# Flower Classification Model

In [6]:
import matplotlib.pyplot as plt

import numpy as np

import os

from PIL import Image

import tensorflow as tf

In [3]:
categories = sorted(os.listdir("./data/flowers/train"))

print("--------------------------------------------------")
print("Categories")
print("--------------------------------------------------")
for category in categories:
    print(category)
print("--------------------------------------------------")

--------------------------------------------------
Categories
--------------------------------------------------
daisy
dandelion
rose
sunflower
tulip
--------------------------------------------------


In [27]:
IMAGE_SIZE = 64
PADDING_COLOR = (0, 0, 0)


def load_image(path):
    image = Image.open(path)
    ratio = image.size[0] / image.size[1]
    size = (
        (IMAGE_SIZE, 2 * round(IMAGE_SIZE / ratio / 2))
        if ratio > 1
        else (2 * round(IMAGE_SIZE * ratio / 2), IMAGE_SIZE)
    )
    left = round((IMAGE_SIZE - size[0]) / 2)
    top = round((IMAGE_SIZE - size[1]) / 2)
    new_image = Image.new(image.mode, (IMAGE_SIZE, IMAGE_SIZE), PADDING_COLOR)
    new_image.paste(image.resize(size), (left, top))
    return np.array(new_image)

In [36]:
ds_train = []

TRAIN_DIR = "./data/flowers/train/"

for index, category in enumerate(categories):
    files = os.listdir(TRAIN_DIR + category)
    for file in files[:1]:
        image = load_image(TRAIN_DIR + category + "/" + file)
        label = index
        print(image)
        print(label)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
1
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  